In [2]:
!pip install opencv-python
!pip install tensorflow  
!pip install numpy  
!pip install pandas  
!pip install keras  
!pip install adam  

In [3]:
!pip install cinit

In [4]:
import sys, os  
import pandas as pd  
import numpy as np  

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [6]:
from keras.utils import to_categorical

In [7]:
df = pd.read_csv(r'C:\Users\user\Documents\fer2013.csv')

In [8]:
X_train,train_y,X_test,test_y=[],[],[],[]  
  
for index, row in df.iterrows():  
    val=row['pixels'].split(" ")  
    try:  
        if 'Training' in row['Usage']:  
           X_train.append(np.array(val,'float32'))  
           train_y.append(row['emotion'])  
        elif 'PublicTest' in row['Usage']:  
           X_test.append(np.array(val,'float32'))  
           test_y.append(row['emotion'])  
    except:  
        print(f"error occured at index :{index} and row:{row}")  
  
  
num_features = 64  
num_labels = 7  
batch_size = 64  
epochs = 30  
width, height = 48, 48

In [9]:
X_train = np.array(X_train,'float32')  
train_y = np.array(train_y,'float32')  
X_test = np.array(X_test,'float32')  
test_y = np.array(test_y,'float32')  
  
train_y = to_categorical(train_y, num_classes=num_labels)
test_y = to_categorical(test_y, num_classes=num_labels)

In [10]:
from tensorflow.keras.utils import to_categorical

In [11]:
X_train -= np.mean(X_train, axis=0)  
X_train /= np.std(X_train, axis=0)

In [12]:
X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)  
  
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)  
  
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [13]:
model = Sequential()  

In [14]:
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))  
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))

In [15]:
#2nd convolution layer  
model.add(Conv2D(64, (3, 3), activation='relu'))  
model.add(Conv2D(64, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))

In [16]:
model.add(Conv2D(128, (3, 3), activation='relu'))  
model.add(Conv2D(128, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

In [17]:
model.add(Flatten()) 

In [18]:
#fully connected neural networks  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
  
model.add(Dense(num_labels, activation='softmax'))

In [19]:
model.compile(loss=categorical_crossentropy,  
              optimizer=Adam(),  
              metrics=['accuracy'])

In [20]:
model.fit(X_train, train_y,  
          batch_size=batch_size,  
          epochs=epochs,  
          verbose=1,  
          validation_data=(X_test, test_y),  
          shuffle=True)

Epoch 1/30


KeyboardInterrupt: 

In [31]:
import os  
import cv2  
import numpy as np  
from keras.models import model_from_json  
from tensorflow.keras.utils import img_to_array
#from keras.preprocessing import image (原本的)
import time

In [22]:
model.save("my_model.h5")

In [23]:
from keras.models import load_model

model = load_model("my_model.h5")

In [24]:
face_haar_cascade = cv2.CascadeClassifier('C:/Users/user/anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')

In [25]:
cap=cv2.VideoCapture(0)  

In [34]:


while True:
    ret, test_img = cap.read()  # 讀取影像
    if not ret:
        break  # 如果影像讀取失敗，跳出迴圈

    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  # 轉為灰階
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
    
    for (x, y, w, h) in faces_detected:
        cv2.rectangle(test_img, (x, y), (x+w, y+h), (255, 0, 0), thickness=7)
    
    # 顯示影像
    cv2.imshow('Face Detection', test_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [37]:
for (x, y, w, h) in faces_detected:
    cv2.rectangle(test_img, (x, y), (x+w, y+h), (255, 0, 0), thickness=7)
    roi_gray = gray_img[y:y+h, x:x+w]  # 修正索引錯誤 (y:y+h, x:x+w)
    roi_gray = cv2.resize(roi_gray, (48, 48))
    
    # 正確使用 img_to_array
    img_pixels = img_to_array(roi_gray)
    img_pixels = np.expand_dims(img_pixels, axis=0)
    img_pixels /= 255

In [44]:
# 虛構的上一行程式碼
predictions = model.predict(img_pixels)  

# find max indexed array  
max_index = np.argmax(predictions[0])  

1/1 [==============================] - 0s 61ms/step


In [45]:
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')  
        predicted_emotion = emotions[max_index] 

In [46]:
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  

array([[[ 56,  58,  51],
        [ 55,  58,  51],
        [ 64,  68,  60],
        ...,
        [ 84,  92,  95],
        [ 85,  93,  96],
        [ 84,  92,  95]],

       [[ 59,  62,  55],
        [ 48,  53,  45],
        [ 56,  62,  55],
        ...,
        [ 84,  92,  95],
        [ 84,  92,  95],
        [ 86,  94,  98]],

       [[ 53,  56,  49],
        [ 52,  57,  51],
        [ 55,  63,  57],
        ...,
        [ 86,  94,  98],
        [ 85,  93,  96],
        [ 87,  95,  99]],

       ...,

       [[ 80,  74,  51],
        [ 79,  70,  47],
        [ 85,  72,  51],
        ...,
        [146, 160, 167],
        [145, 161, 167],
        [145, 161, 167]],

       [[ 97,  91,  64],
        [ 83,  74,  49],
        [ 89,  78,  55],
        ...,
        [146, 160, 167],
        [141, 157, 164],
        [143, 159, 166]],

       [[ 80,  73,  44],
        [ 79,  71,  44],
        [ 83,  74,  50],
        ...,
        [146, 160, 167],
        [142, 158, 165],
        [142, 158, 165]]

In [47]:
    resized_img = cv2.resize(test_img, (1000, 700))  
    cv2.imshow('Facial emotion analysis ',resized_img)

In [49]:
    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed  
        'break' 

In [50]:
cap.release()  
cv2.destroyAllWindows

<function destroyAllWindows>

In [ ]:
#這東西太邪門了